## This notebook performs a single visit for this run for a target XXXYYY
##### The script, slews and acquires the target, applies focus correction, takes sequence of data

#### WARNING: This currently requires ts_externalscripts branch **tickets/DM-28818**

In [ ]:
#TARGET='HD 60753' # setting
#TARGET="HD 34816" #pole 
class target():
    name = "HD 60753"
    mag = 6.65

In [ ]:
print(f'Selected {target.name} with a magnitude of f{target.mag}')

In [ ]:
import os
import sys
import asyncio
import time

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

from lsst.ts.idl.enums.Script import ScriptState

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
# for autocompleted to work
%config IPCompleter.use_jedi = False

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [ ]:
# make sure all remotes etc are running
script = LatissAcquireAndTakeSequence(index=1)  # this essentially calls the init method of the script
await script.start_task

In [ ]:
# set wrap strategy
# this is required until the ATPtg is updated to not configure the mount for maximum time on target
script.atcs.rem.atptg.cmd_raDecTarget.set(azWrapStrategy=1)  # 1 does not unwrap, 0 unwraps

In [ ]:
base_mag_acq_exp_time = 0.5
base_mag=5.5
base_mag_seq_exp_time = 15

In [ ]:
# scale exposure times to magnitude
acq_exposure_time = base_mag_acq_exp_time * 10.0**((target.mag-base_mag)/2.5) # seconds
seq_exposure_time = base_mag_seq_exp_time * 10.0**((target.mag-base_mag)/2.5)

In [ ]:
# Impose lower limits on exposure times
if acq_exposure_time < 0.3:
    acq_exposure_time = 0.3
if seq_exposure_time < 5.0:
    seq_exposure_time = 5.0

In [ ]:
print(f'Acquisition exposure time is {acq_exposure_time:0.2f} [s]')
print(f'Ronchi exposure time is {seq_exposure_time:0.2f} [s]')

In [ ]:
exposure_time_sequence = [seq_exposure_time]*7
exposure_time_sequence.append(1.0)

In [ ]:
seq_yaml = yaml.safe_dump({"object_name": target.name,
                                "do_acquire": True,
                                "acq_filter" : 'RG610',
                                "acq_grating" : 'empty_1', 
                                "acq_exposure_time": acq_exposure_time ,
                                "max_acq_iter": 3,
                                "do_take_sequence": True,
#                                 "exposure_time_sequence" : [15,15,15,15,
#                                                             15,15,15, 1], 
                                "exposure_time_sequence" : exposure_time_sequence, 
                                "filter_sequence": ['RG610', 'RG610', 'RG610', 'RG610',
                                                    'RG610', 'empty_1', 'empty_1', 'RG610'], 
                                    # RG610 and Ronchi
                                "grating_sequence": ['ronchi90lpmm', 'ronchi90lpmm','ronchi90lpmm','ronchi90lpmm',
                                                     'ronchi90lpmm', 'empty_1', 'empty_1', 'empty_1'],
                                "do_pointing_model": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "target_pointing_verification": False})
print(seq_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
seq_configuration = script.cmd_configure.DataType()
seq_configuration.config = seq_yaml
await script.do_configure(seq_configuration)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# get group_id_data that will be used for all images for this target
time_for_groupID=astropy.time.Time.now().isot
group_id_data = script.cmd_setGroupId.DataType(
                groupId=time_for_groupID
            )
await script.do_setGroupId(group_id_data)

In [ ]:
# run acquisition script
run_acq = script.cmd_run.DataType()
await script.do_run(run_acq)
await script.done_task

In [ ]:
await script.atcs.stop_tracking()